# [Bonus Tasks](svm_segm.ipynb)

In [1]:

import numpy as np
import matplotlib.pyplot as plt

import scipy.ndimage as ndi
import skimage.morphology as morph
import skimage.util
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [2]:
def blend_result(img, seg, border_radius=1, interior_opacity=1, interior_opacity_decay=0.9, color=(0,1,0)):
    img  = np.dstack([img] * 3).copy()
    img -= img.min()
    img /= img.max()
    selem  = morph.disk(border_radius)
    seg_bd = np.logical_xor(morph.binary_dilation(seg, selem), morph.binary_erosion(seg, selem))
    mask_decay = ndi.distance_transform_edt(seg)
    for i in range(3):
        opacity = interior_opacity / pow(1 + mask_decay[seg], interior_opacity_decay)
        img[:,:,i][seg] = color[i] * opacity + (1 - opacity) * img[:,:,i][seg]
        img[:,:,i][seg_bd] = color[i]
    return img

In [12]:
patch_sizes = (32, 32)

---

**[Task 1.1.]()** Implement `create_data_matrix`:

In [29]:
def create_data_matrix(img, patch_sizes):
    #Step 1: creating nonoverlapping patches
    patches=skimage.util.view_as_blocks(img, patch_sizes)

    #Step 2: Flatten each block und reshape the output into 2D matrix
    num_patches= patches.shape[0] * patches.shape [1] #number of patches
    patch_height, patch_width = patch_sizes

    #Flatten the patches and reshape into a 2D matrix
    X = patches.reshape (num_patches, patch_height * patch_width)
    return X
    


**[Task 1.2.]()** Implement `create_gt_labels_vector`:

In [30]:
def create_gt_labels_vector(gt_img, patch_sizes):
    # Step 1: Create non-overlapping patches
    patches = skimage.util.view_as_blocks(gt_img, block_shape=patch_sizes)

    # Step 2: Flatten patches and compute labels
    num_patches_y, num_patches_x, patch_height, patch_width = patches.shape

    # Flatten the patches and calculate the labels
    labels = np.zeros(num_patches_y * num_patches_x, dtype=int)

    # Iterate through the patches and calculate the label for each patch
    for i in range(num_patches_y):
        for j in range(num_patches_x):
            patch = patches[i, j]
            # Calculate the percentage of foreground pixels
            foreground_percentage = np.sum(patch) / (patch_height * patch_width)
            
            # Assign the label based on the percentage of foreground
            if foreground_percentage > 0.5:
                labels[i * num_patches_x + j] = 1  # More than 50% foreground
            elif foreground_percentage == 0:
                labels[i * num_patches_x + j] = -1  # No foreground
            else:
                labels[i * num_patches_x + j] = 0  # Mixed foreground and background

    return labels

**[Task 1.3.]()** Create the SVM classifier:

In [31]:
clf = make_pipeline(StandardScaler(), SVC(class_weight='balanced', gamma=0.1))

**[Task 1.3 (a).]()** Create the data matrices for the images `dna-33` and `dna-44`:

In [33]:
img_33 = plt.imread("data/NIH3T3/gt/33.png")
img_44 = plt.imread("data/NIH3T3/gt/44.png")
img_33_dm = create_data_matrix(img_33, patch_sizes)
img_44_dm = create_data_matrix(img_44, patch_sizes)

**[Task 1.3 (b).]()** Create the corresponding ground truth label vectors:

In [36]:

img_33_labels = create_gt_labels_vector(img_33, patch_sizes)
img_44_labels = create_gt_labels_vector(img_44, patch_sizes)


**[Task 1.3 (c).]()** Create the *combined* data matrices and ground truth label vectors:

In [45]:

def stack_data_matrices_and_labels(img_33_dm, img_33_labels, img_44_dm, img_44_labels):
    # Concatenate data matrices along axis 0 (rows)
    X_stacked = np.concatenate((img_33_dm, img_44_dm), axis=0)
    
    # Concatenate label vectors along axis 0 (rows)
    y_stacked = np.concatenate((img_33_labels, img_44_labels), axis=0)
    
    return X_stacked, y_stacked

    
print("Shape of stacked data matrix:", X_stacked.shape)
print("Shape of stacked label vector:", y_stacked.shape)

Shape of stacked data matrix: (2109, 1024)
Shape of stacked label vector: (2109,)


**[Task 1.3 (d).]()** Train the classifier using the data matrix and label vectors from above:

In [39]:
clf_trained=clf.fit(X_stacked, y_stacked)

**[Task 1.4.]()** Implement the function `predict_image`:

In [52]:
def predict_image(img):
    # Step 1: Use create_data_matrix to get the data matrix for the image
    X = create_data_matrix(img, patch_sizes)
    
    # Step 2: Predict the labels for each patch using the trained classifier
    predicted_labels = clf.predict(X)
    
    # Step 3: Create the binary result image
    # Calculate the number of patches along x and y axes
    num_patches_y = img.shape[0] // patch_sizes[0]  # Number of patches in height
    num_patches_x = img.shape[1] // patch_sizes[1]  # Number of patches in width
    
   # Step 4: Reshape predicted_labels into a 2D array corresponding to patches in the image
    predicted_labels_reshaped = predicted_labels.reshape(num_patches_y, num_patches_x)
    
    # Step 5: Create a binary result image: True for foreground patches, False otherwise
    result = predicted_labels_reshaped == 1  # Set foreground patches as True, background as False
    
    # Step 6: Return the result as a binary image
    return result



Test your implementation:

In [53]:
img = plt.imread(f'data/NIH3T3/im/dna-0.png')
seg = predict_image(img)

plt.figure()
plt.imshow(blend_result(img, seg))

IndexError: boolean index did not match indexed array along dimension 0; dimension is 1024 but corresponding boolean dimension is 32

<Figure size 640x480 with 0 Axes>

**[Task 1.5.]()** Perform batch processing: